In [1]:
import requests
import os
import csv
import json
import time
import dateutil.parser
import pandas as pd
from tqdm import tqdm
import datetime
import unicodedata

C:\Users\nadeg\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\nadeg\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\nadeg\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
bearer_token='AAAAAAAAAAAAAAAAAAAAAHXvUQEAAAAAPQ%2FXl8vv2ZxuPlNEMn%2BbAvOg428%3DCNmiqFmvchzJIzzhLvyFBvL1mnOuwGjALg6uSbECpZIojXxUIM'
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAHXvUQEAAAAAPQ%2FXl8vv2ZxuPlNEMn%2BbAvOg428%3DCNmiqFmvchzJIzzhLvyFBvL1mnOuwGjALg6uSbECpZIojXxUIM'

In [27]:
class DataLoader:
    '''
    Collects data from Twitter Academic API endpoints and provides them as json or csv files. Requires a bearer token for authentication.
    '''
    def __init__(self, bearer_token):
        self.bearer_token = bearer_token
        self.headers = {"Authorization": "Bearer {}".format(self.bearer_token)}
        
        '''
        Initialize query parameters for the Twitter Full Archive Search endpoint.
        Args:
            start_date (str): the start time of the period. It needs to be a valid timestamp.
            end_data (str): the end time of the period. It needs to be a valid timestamp.
            keyword (str): the query parameters to refine the tweet search. See the Twitter API documentation for more information on queries.
            max_results (int): maximum number of tweets to retrieve for the desired period
        Returns:
            query_params (dict): dictionary containing all parameters for the Archive Search query.            
        '''
        
    def create_tweet_query(self,
                     start_date='2020-01-01 T00:00:00.000Z', 
                     end_date='2020-02-01 T00:00:00.000Z',
                     keyword="place_country:BE has:geo lang:nl",
                     max_results = 500):
        query_params = {'query': keyword,
                        'start_time': start_date,
                        'end_time': end_date,
                        'max_results': max_results,
                        'expansions': 'author_id,in_reply_to_user_id,geo.place_id,referenced_tweets.id',
                        'tweet.fields': 'id,text,author_id,context_annotations,geo,created_at,lang,public_metrics,entities,reply_settings,possibly_sensitive,source',
                        'user.fields': 'id,name,username,created_at,description,location,public_metrics,verified,entities,profile_image_url',
                        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                        'next_token': {}}
        return query_params
    
        
    def connect_to_endpoint(self, search_api, query_params, next_token = None):
        '''
        Establish connection with the Twitter API endpoint.
        '''
        query_params['next_token'] = next_token   
        response = requests.request("GET", search_api, headers = self.headers, params = query_params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return response.json()
    
    def retrieve_tweet(self, start_list, end_list, keyword="place_country:BE has:geo lang:nl", tweet_per_period = 10000):
        '''
        Retrieve tweets from the Full Archive Search, following a specific query, for a given number of tweets per time periods.
        Args:
            start_list (list): list containing the start times for each time period. Needs to have the same length as end_list.
            end_list (list): list containing the end times for each time period. Needs to have the same length as start_list.
            keyword (str):  the query parameters to refine the tweet search. See the Twitter API documentation for more information on queries.
            tweet_per_period (int): the maximum number of tweets to retrieve per time period.
        Returns:
            tweet_list (list): list containing all the Twitter response objects.
        '''
        tweets_list = []  
        #Define the search API : Twitter Full Archive Search
        search_api = "https://api.twitter.com/2/tweets/search/all"
        #Total number of tweets we collected from the loop
        total_tweets = 0
        #Ensures that we collect less than or equal to the desired number of tweets
        #Define the number of results per query. The API accepts a maximum of 500 tweets per query.
        if tweet_per_period >= 500:
            max_results=500
        else:
            max_results=tweet_per_period
        for i in range(0,len(start_list)):
            count = 0 # Counting tweets per time period
            #Max tweets per period is set a bit lower than the real objective
            flag = True
            next_token = None
       
            while flag:
                # Check if max_count reached
                if count >= tweet_per_period:
                    break
                print("-------------------")
                print("Token: ", next_token)
                query = self.create_tweet_query(start_list[i],end_list[i],keyword,max_results)
                json_response = self.connect_to_endpoint(search_api, query, next_token)
                result_count = json_response['meta']['result_count']

                if 'next_token' in json_response['meta']:
                    # Save the token to use for next call
                    next_token = json_response['meta']['next_token']
                    print("Next Token: ", next_token)
                    if result_count != None and result_count > 0 and next_token != None:
                        print("Start Date: ", start_list[i])
                        tweets_list.append(json_response)
                        count += result_count
                        total_tweets += result_count
                        print("Total # of Tweets added: ", total_tweets)
                        print("-------------------")
                        time.sleep(3)

                # If no next token exists
                else:
                    if result_count != None and result_count > 0:
                        print("-------------------")
                        print("Start Date: ", start_list[i])
                        tweets_list.append(json_response)
                        count += result_count
                        total_tweets += result_count
                        print("Total # of Tweets added: ", total_tweets)
                        print("-------------------")
                        time.sleep(3)

                    #Since this is the final request, turn flag to false to move to the next time period.
                    flag = False
                    next_token = None
                time.sleep(3)
        print("Total number of results: ", total_tweets)
        
        return tweets_list
    
    
    def load_json(self, file_path='data/raw_tweets/tweet_dataset.json'):
        
        '''
        Load a Twitter response json file.
        '''
        with open(file_path) as json_file:
            return json.load(json_file)
   

    def get_info(self,tweets):
        
        '''
        Convert the json raw data to a tabular Pandas DataFrame.
        Args:
            data (json):raw data collected from the Twitter API
        Returns:
            text_list: list of texts from tweets
            tweet_df (pd.DataFrame) : DataFrame with tweet text and metadata
       '''
        
    #tweet
        
        text = []

        for i in tqdm(range(len(tweets))):
            tweet_content = tweets[i]['data']
            #tweet
            for t in range(len(tweet_content)) : 
                
                text.append(tweet_content[t]['text'] )
        tweet_df = pd.DataFrame({"text":text})   
        return text, tweet_df

In [40]:
from twitter_sentiment_classifier import batch_predict
dl = DataLoader(bearer_token)   

#Retrieve around 200 tweets, 100 on 2020/01/01 and 100 on 2020/01/02, written in English
tweet_list = dl.retrieve_tweet(start_list=['2020-01-01T00:00:00.000Z','2020-02-01T00:00:00.000Z'],
                               end_list=['2020-01-31T00:00:00.000Z','2020-02-28T00:00:00.000Z'],
                                keyword="overheid place_country:BE has:geo lang:nl",
                               tweet_per_period=100, 
                               )

tekst_list, tekst_df = dl.get_info(tweet_list)
tekst_df.to_csv('output_info.csv')
for i in range(0,len(tekst_list)):
    #preprocessing stap: vervangen van einde lijn naar spatie
    tekst_list[i] = tekst_list[i].replace("\n"," ")
    
batch_predict(tekst_list)


-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-01-01T00:00:00.000Z
Total # of Tweets added:  84
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2020-02-01T00:00:00.000Z
Total # of Tweets added:  180
-------------------
Total number of results:  180


Predicting...: 100%|██████████| 6/6 [00:34<00:00,  5.74s/it]


['NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'NEGATIVE',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NEUTRAL',
 'NEGATIVE',
 'NEUTRAL',
 'NEUTRAL',
 'NE